In [10]:
import numpy as np
import pandas as pd
from IPython.display import display

In [6]:
data_2013 = pd.read_csv("./data/2012-2013 Solar home electricity data v2.csv", header = 1)

In [26]:
data_2013

,Customer,Generator Capacity,Postcode,Consumption Category,date,0:30,1:00,1:30,2:00,2:30,...,20:00,20:30,21:00,21:30,22:00,22:30,23:00,23:30,0:00,Row Quality
0,1,3.78,2076,CL,1/07/2012,1.250,1.250,1.250,1.263,0.131,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.081,NaN
1,1,3.78,2076,GC,1/07/2012,0.855,0.786,0.604,0.544,0.597,...,0.374,0.447,0.549,0.136,0.288,0.181,0.651,0.090,0.068,NaN
2,1,3.78,2076,GG,1/07/2012,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,NaN
3,1,3.78,2076,CL,2/07/2012,1.250,1.250,1.125,0.000,0.925,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.069,NaN
4,1,3.78,2076,GC,2/07/2012,0.309,0.082,0.059,0.097,0.290,...,0.353,0.464,0.229,0.811,0.222,0.306,1.034,0.136,0.067,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268552,300,3.36,2086,GC,29/06/2013,0.171,0.832,0.440,0.745,0.149,...,0.383,0.404,0.367,0.398,0.350,0.228,0.170,0.139,0.171,NaN
268553,300,3.36,2086,GG,29/06/2013,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,NaN
268554,300,3.36,2086,CL,30/06/2013,2.501,2.475,2.491,2.508,2.490,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2.468,NaN
268555,300,3.36,2086,GC,30/06/2013,0.150,0.158,0.157,0.137,0.163,...,0.625,0.595,0.695,0.646,0.581,0.458,0.472,0.732,0.297,NaN


In [77]:
data_2013.columns[0]

'Customer'

In [28]:
#count the number of NA (non-actual) in 'Row Quality' column:
print(f"number of NA (non-actual) row: {data_2013['Row Quality'].count()}")

number of NA (non-actual) row: 0


In [69]:
data_cust = data_2013.groupby(by='Customer')
cust_info = pd.DataFrame()
cust_info['Customer ID'] = data_cust['Customer'].first()
cust_info['Generator Capacity'] = data_cust['Generator Capacity'].first()
cust_info['Postcode'] = data_cust['Postcode'].first()
list_cust = [c for _,c in data_cust]
cust_type = [c.groupby(by='Consumption Category') for c in list_cust]
cust_GC = [t.get_group('GC') for t in cust_type]

300


In [133]:
meltdate = data_2013.melt(id_vars = ['Customer', 'Consumption Category', 'date'], value_vars = data_2013.columns[5:53], var_name = 'time', value_name = 'power')
stk = meltdate.set_index(['Customer', 'date', 'time']).stack()
#melt = data_2013.melt(id_vars = ['Customer', 'Consumption Category'], value_vars = data_2013.columns[4:53], var_name = 'time', value_name = 'power')
#cat = [c.groupby(by='time').sum() for _,c in melt.groupby(by = 'Consumption Category')]
#cat[0]

In [140]:
meltdate['timestamp'] = meltdate['date'].astype(str) + meltdate['time'].astype(str)

In [ ]:
unstk.head()

In [128]:
meltdate.head(10)

,Customer,Consumption Category,date,time,power
0,1,CL,1/07/2012,0:30,1.250
1,1,GC,1/07/2012,0:30,0.855
2,1,GG,1/07/2012,0:30,0.000
3,1,CL,2/07/2012,0:30,1.250
4,1,GC,2/07/2012,0:30,0.309
5,1,GG,2/07/2012,0:30,0.000
6,1,CL,3/07/2012,0:30,1.244
7,1,GC,3/07/2012,0:30,0.092
8,1,GG,3/07/2012,0:30,0.000
9,1,CL,4/07/2012,0:30,1.256


In [136]:
stk.reset_index()

,Customer,date,time,level_3,0
0,1,1/07/2012,0:30,Consumption Category,CL
1,1,1/07/2012,0:30,power,1.25
2,1,1/07/2012,0:30,Consumption Category,GC
3,1,1/07/2012,0:30,power,0.855
4,1,1/07/2012,0:30,Consumption Category,GG
...,...,...,...,...,...
25781467,300,30/06/2013,0:00,power,2.468
25781468,300,30/06/2013,0:00,Consumption Category,GC
25781469,300,30/06/2013,0:00,power,0.297
25781470,300,30/06/2013,0:00,Consumption Category,GG


In [123]:
piv = meltdate.loc[:10].pivot(index = "time", columns = "Consumption Category", values = "power")

ValueError: Index contains duplicate entries, cannot reshape